# Hello World

## Simple invoke

In [1]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="deepseek-v3.1:671b-cloud",
    validate_model_on_init=True,
    temperature=0.8,
    num_predict=256,
    # other params ...
)

messages = [
    ("system", "You are a helpful translator. Translate the user sentence to French."),
    ("human", "I love programming."),
]
model.invoke(messages)

c:\Users\facun\anaconda3\envs\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AIMessage(content="J'adore la programmation.", additional_kwargs={}, response_metadata={'model': 'deepseek-v3.1:671b-cloud', 'created_at': '2025-11-16T20:45:50.261373914Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1032841066, 'load_duration': None, 'prompt_eval_count': 21, 'prompt_eval_duration': None, 'eval_count': 8, 'eval_duration': None, 'model_name': 'deepseek-v3.1:671b-cloud', 'model_provider': 'ollama'}, id='lc_run--ac876085-9f49-4661-bba9-e2005cb5c774-0', usage_metadata={'input_tokens': 21, 'output_tokens': 8, 'total_tokens': 29})

## Streaming invoke

In [2]:
for chunk in model.stream("Return the words Hello World! separatedly."):
    print(chunk.text, end="")

Here are the words "Hello World!" separated individually:

**Hello**  
**World!**

## Asynchronous invoke

In [3]:
await model.ainvoke("Hello how are you!")

async for chunk in model.astream("Say hello world!"):
    print(chunk.content)

Hello world
!
 😊





## Batch invoke

In [4]:
messages = [("human", "Say hello world!"), ("human", "Say goodbye world!")]
await model.abatch(messages)

[AIMessage(content='Hello, world! 🌍', additional_kwargs={}, response_metadata={'model': 'deepseek-v3.1:671b-cloud', 'created_at': '2025-11-01T23:39:00.883273419Z', 'done': True, 'done_reason': 'stop', 'total_duration': 651696330, 'load_duration': None, 'prompt_eval_count': 10, 'prompt_eval_duration': None, 'eval_count': 7, 'eval_duration': None, 'model_name': 'deepseek-v3.1:671b-cloud', 'model_provider': 'ollama'}, id='lc_run--823925dc-9c18-4534-af3d-41f44c88dfb1-0', usage_metadata={'input_tokens': 10, 'output_tokens': 7, 'total_tokens': 17}),
 AIMessage(content='Goodbye, world! 👋  \nIf you have any more questions or need help in the future, I’m here for you. Take care! 😊', additional_kwargs={}, response_metadata={'model': 'deepseek-v3.1:671b-cloud', 'created_at': '2025-11-01T23:39:01.207991262Z', 'done': True, 'done_reason': 'stop', 'total_duration': 963045650, 'load_duration': None, 'prompt_eval_count': 10, 'prompt_eval_duration': None, 'eval_count': 34, 'eval_duration': None, 'model

## Json format response

In [5]:
json_model = ChatOllama(model="deepseek-v3.1:671b-cloud", format="json")
res = json_model.invoke(
    "Return a query for the weather in a random location and time of day with two keys: location and time_of_day. "
    "Respond using JSON only."
)

print(res.content)

```json
{
  "location": "Kyoto",
  "time_of_day": "sunset"
}
```


## Tool use example

In [6]:
from typing import List
from langchain_ollama import ChatOllama
from langchain.tools import tool

@tool
def validate_user(user_id: int, addresses: List[str]) -> bool:
    """Validate user using historical addresses.

    Args:
        user_id (int): the user ID.
        addresses (List[str]): Previous addresses as a list of strings.
    """
    return user_id == 42

model = ChatOllama(
    model="deepseek-v3.1:671b-cloud",
    temperature=0,
).bind_tools([validate_user])

ans = model.invoke("Could you validate user 123? They previously lived at "
    "123 Fake St in Boston MA and 234 Pretend Boulevard in "
    "Houston TX. Tell me if they are valid."
)

print(ans.tool_calls)
print(ans)

[]
content='I can help you validate user 123 using their previous addresses. However, I need to clarify something about the address format you provided.\n\nYou mentioned "123 Fake St in Boston MA" and "234 Pretend Boulevard in Houston TX" - should I use these addresses exactly as written, or would you prefer them formatted in a more standard address format (like "123 Fake St, Boston, MA" and "234 Pretend Boulevard, Houston, TX")?\n\nOnce you confirm the preferred format, I\'ll proceed with the validation using the validate_user function.' additional_kwargs={} response_metadata={'model': 'deepseek-v3.1:671b-cloud', 'created_at': '2025-11-01T23:39:06.907002371Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3626433136, 'load_duration': None, 'prompt_eval_count': 233, 'prompt_eval_duration': None, 'eval_count': 112, 'eval_duration': None, 'model_name': 'deepseek-v3.1:671b-cloud', 'model_provider': 'ollama'} id='lc_run--bdf0d43f-438c-4104-ae1d-33a5497ed772-0' usage_metadata={'inp

In [7]:
from langchain.messages import AIMessage

if isinstance(ans, AIMessage) and ans.tool_calls:
    print(ans.tool_calls)

## Prompt templates example

In [8]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate

template = PromptTemplate.from_template("""Answer the question based on the context below.
    If the question can't be answered using the context, say "I don't know".

    Context: {context}

    Question: {question}

    Answer:""")

model = ChatOllama(
    model="deepseek-v3.1:671b-cloud",)

prompt = template.invoke({
    "context": "The most recent advancements in AI technology have led to significant improvements in natural language processing.",
    "question": "Which model providers offer LLMs?"
})

completion = model.invoke(prompt)

print(completion)

content="I don't know." additional_kwargs={} response_metadata={'model': 'deepseek-v3.1:671b-cloud', 'created_at': '2025-11-01T23:39:07.910810382Z', 'done': True, 'done_reason': 'stop', 'total_duration': 596445556, 'load_duration': None, 'prompt_eval_count': 65, 'prompt_eval_duration': None, 'eval_count': 6, 'eval_duration': None, 'model_name': 'deepseek-v3.1:671b-cloud', 'model_provider': 'ollama'} id='lc_run--9a47ef06-cf4f-4986-b99c-6251f53012ed-0' usage_metadata={'input_tokens': 65, 'output_tokens': 6, 'total_tokens': 71}


## Chain example

In [9]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that provides answers with justifications. You must respond in JSON format using the specified keys."),
    ("human", "{question}"),
])

model = ChatOllama(
    model="gpt-oss:120b-cloud",
    temperature=0,
)

parser = JsonOutputParser(keys=["answer", "justification"])

chain = prompt | model | parser

response = chain.invoke(
    {"question": """What weight more, a pound of feathers or a pound of bricks? (use plain text)"""}
)

import json

print(json.dumps(response, indent=2))

{
  "answer": "They weigh the same.",
  "justification": "A pound is a unit of weight. Whether the material is feathers or bricks, one pound of each contains the same weight of 1 pound."
}
